## Playground 3

Workshopping LLM Output Code Execution and Advanced RAG

In [41]:
import pandas as pd
import numpy as np
import json, os, pprint
import matplotlib.pyplot as plt
import plotly.express as px
import random
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser, JsonOutputKeyToolsParser
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent, Tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools import PythonREPLTool
from langchain import hub
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.callbacks import Callbacks
from langchain.prompts import ChatPromptTemplate

In [2]:
os.environ["OPENAI_API_KEY"] = ""

Creating a Python execution tool

In [16]:
python_repl = PythonREPL()

python_repl.run("import numpy as np; x=np.random.rand(3,2); print(x)")

'[[0.06119131 0.76473527]\n [0.82000326 0.26274933]\n [0.85977688 0.88628719]]\n'

In [3]:
python_repl = PythonREPL()

exec_python_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [4]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1, streaming=True)

In [29]:
llm_with_tools = llm.bind_tools([exec_python_tool])

python_create_chain = llm_with_tools | JsonOutputToolsParser()

In [30]:
d = python_create_chain.invoke("can you generate a random vector in python of size 3 by 2?")
d

[{'type': 'python_repl',
  'args': {'__arg1': 'import numpy as np\n\n# Generate a random vector of size 3x2\nrandom_vector = np.random.rand(3, 2)\n\nprint(random_vector)'}}]

In [33]:
d[0]["args"]["__arg1"]

'import numpy as np\n\n# Generate a random vector of size 3x2\nrandom_vector = np.random.rand(3, 2)\n\nprint(random_vector)'

**Use with a more specific template**

In [35]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

Since it is in markdown we need to clean the output

In [36]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

In [39]:
python_create_chain_2 =  prompt | llm | StrOutputParser() | _sanitize_output | PythonREPL().run

python_create_chain_2.invoke(
    {
        "input":"can you generate a random vector in python of size 3 by 2?"
    }
)

'[[0.43141703 0.19956448]\n [0.26696694 0.17928585]\n [0.59987539 0.47697729]]\n'

**That's great, but we probably need a way to ask the user to look at the code an confirm it before running for more complex uses cases**

In [45]:
tools = [PythonREPLTool()]

In [46]:
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [58]:
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [62]:
agent_executor.invoke(
        {
        "input": "When was the first Covid-19 case reported?"
        }
    )



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `print('December 31, 2019')`


December 31, 2019
The first Covid-19 case was reported on December 31, 2019.

> Finished chain.


{'input': 'When was the first Covid-19 case reported?',
 'output': 'The first Covid-19 case was reported on December 31, 2019.'}